In [1]:
!samtools view ../datapack/CLIP-35L33G.bam | head -6

SRR458758.23028115	0	chr1	3056473	0	20M	*	0	0	GAATGGAAGTTCAAGGATCT	HHGHHGE@GGHHED?GEGDG	MD:Z:20	NH:i:40	HI:i:1	NM:i:0	SM:i:0	XQ:i:40	X2:i:40	XO:Z:UM	XS:A:-
SRR458758.23879202	0	chr1	3056473	0	20M	*	0	0	GAATGGAAGTTCAAGGATCT	GBGGEEAAF=CEEDEGBGG>	MD:Z:20	NH:i:40	HI:i:1	NM:i:0	SM:i:0	XQ:i:40	X2:i:40	XO:Z:UM	XS:A:-
SRR458758.23893532	0	chr1	3056473	0	20M	*	0	0	GAATGGAAGTTCAAGGATCT	G@?GGE=EFBDDGGGGD@GG	MD:Z:20	NH:i:40	HI:i:1	NM:i:0	SM:i:0	XQ:i:40	X2:i:40	XO:Z:UM	XS:A:-
SRR458758.26501547	0	chr1	3056473	0	20M	*	0	0	GAATGGAAGTTCAAGGATCT	IIIHIIIHIGHIIIHEHIHH	MD:Z:20	NH:i:40	HI:i:1	NM:i:0	SM:i:0	XQ:i:40	X2:i:40	XO:Z:UM	XS:A:-
SRR458758.685997	16	chr1	3059006	0	22M	*	0	0	TTCATTTACAGAATGGAATACT	EG7:776DGGGBD<GGEBGEG@	MD:Z:22	NH:i:30	HI:i:1	NM:i:0	SM:i:0	XQ:i:40	X2:i:40	XO:Z:UM	XS:A:+
SRR458758.12045118	16	chr1	3059006	0	22M	*	0	0	TTCATTTACAGAATGGAATACT	IFGIDDIHIHEIIIIIIIIHBI	MD:Z:22	NH:i:30	HI:i:1	NM:i:0	SM:i:0	XQ:i:40	X2:i:40	XO:Z:UM	XS:A:+
samtools view: writing to standard output failed: Broken

In [3]:
!samtools view ../datapack/CLIP-35L33G.bam | \
 awk '{print length($10)}' | sort | uniq -c

1285348 20
1203488 21
1202998 22
1217558 23
1355585 24
1332303 25
1429981 26
1365067 27
1296198 28
1324274 29
1237459 30
1123536 31
1139511 32
1185578 33
1044415 34
1073551 35
1066277 36
1053737 37
1195614 38
1214512 39
1675861 40
800662 41
758539 42
703073 43
710768 44
801734 45
983530 46
608376 47
612593 48
679513 49
840256 50
471328 51
432443 52
399847 53
396185 54
304426 55
295200 56
275327 57
239794 58
252191 59
304281 60
233741 61
225995 62
173815 63
157092 64
146175 65
134258 66
110824 67
103110 68
90087 69
91805 70
85594 71
80883 72
83219 73
121198 74
150140 75


* 일단 `MD:Z:`부분이 mismatch를 표시하고 있기 때문에, 이 부분의 정보를 가지고 read 위치별 error frequency를 구하는 코드를 작성해 본다.
* 전체 CLIPseq reads 의 길이는 20~75 정도이므로, 길이 80인 빈 리스트 `reslist`를 마련해 두고, mdflag에 표기되어있는 error를 순차적으로 추출하여 `reslist`에 저장한다. 각 read의 5'-end 의 첫번째 base 위치가 `reslist`의 0위치에 해당한다.

In [9]:
reslist =[]
for i in range(80):
    reslist.append([])

In [2]:
!samtools view ../datapack/CLIP-35L33G.bam | wc -l

 38880853


* 혹시 모르니 한번 랜덤으로 생성한 MDflag 예제로 테스트를 해본다

In [10]:
import re

# regex pattern to split MDflag by each error
p = re.compile("[0-9]+[\^]*[ACGT]+")

# test to get error information from one MDflag (one line in BAM)
mdflag = "15G4C30^CC0A18" # MDflag example

temp = []
errorI = 0
for err in p.findall(mdflag):
    matchN = int(re.search("[0-9]+",err).group())
    errorI += matchN + 1
    errorB = re.search("[\^ACGT]+", err).group()
    temp.append((errorI, errorB))
temp

[(16, 'G'), (21, 'C'), (52, '^CC'), (53, 'A')]

* 각 라인별로 MDflag를 추출하면 거기서 각각의 error 위치와 뉴크레오타이드를 튜플 리스트로 추출해줄 함수를 작성한다.

In [11]:
def Err_eachMDflag(mdflag):
    p = re.compile("[0-9]+[\^]*[ACGT]+")
    
    temp=[]
    errorI = 0
    for err in p.findall(mdflag):
        matchN = int(re.search("[0-9]+",err).group())
        errorI += matchN + 1
        errorB = re.search("[\^ACGT]+", err).group()
        temp.append((errorI, errorB))
    return temp

* 작성한 함수를 활용하여 결과리스트를 도출하는 과정 테스트

In [19]:
test = ["5G2^CC10", "20", "8A0C8", "0A16"]

testlist =[]
for i in range(20):
    testlist.append([])
    
    
for eachMD in test:
    errList = Err_eachMDflag(eachMD)
    for i, b in errList:
        testlist[i-1].append(b)

testlist

[['A'],
 [],
 [],
 [],
 [],
 ['G'],
 [],
 [],
 ['^CC', 'A'],
 ['C'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 []]

* 원래의 `CLIP-35L33G.bam`파일은 너무 크기 때문에, 가장 작은 chrM만 가지고 연습을 해보기로하였다.

In [6]:
!samtools view -b -o CLIP-chrM.bam ../datapack/CLIP-35L33G.bam chrM
!samtools view CLIP-chrM.bam | head -3

SRR458758.537185	0	chrM	4	39	24M	*	0	0	AATGTGGCTTAATAACAAAGCAAA	BGGGGHHHHHHHHHGBBGGGHDHH	MD:Z:5A18	NH:i:1	HI:i:1	NM:i:1	SM:i:39	XQ:i:40	X2:i:0	XO:Z:UU	XS:A:+	XY:Z:+ENSMUST00000082387.1:4..27
SRR458758.2572947	0	chrM	4	39	24M	*	0	0	AATGTGGCTTAATAACAAAGCAAA	HDHGHHFFHHBBGGGFFDGDDHHH	MD:Z:5A18	NH:i:1	HI:i:1	NM:i:1	SM:i:39	XQ:i:40	X2:i:0	XO:Z:UU	XS:A:+	XY:Z:+ENSMUST00000082387.1:4..27
SRR458758.7266053	0	chrM	4	38	24M	*	0	0	AATGTGGCTTAATAACAAAGCAAA	=D@DGCBDGDGDDDGGEGDADDED	MD:Z:5A18	NH:i:1	HI:i:1	NM:i:1	SM:i:38	XQ:i:40	X2:i:0	XO:Z:UU	XS:A:+	XY:Z:+ENSMUST00000082387.1:4..27
samtools view: writing to standard output failed: Broken pipe
samtools view: error closing standard output: -1


In [22]:
!samtools view -h CLIP-chrM.bam | grep "MD:Z:" | sed 's/.*MD:Z:\([ACGTI^0-9]*\).*/\1/' | head

5A18
5A18
5A18
5A32
5A18
5A18
5A32
5A32
5A32
5A32
sed: stdout: Broken pipe
